# Практика

In [ ]:
from analytics_lib import AnyName

In [ ]:
from analytics_lib import *  # очень нехорошо так делать

In [ ]:
def calculation_2(x, y, z):
    """Просто функция"""
    return x**2 + y**2 + z**2

In [127]:
class AnyName:  # CamelCase snake_case
    BANK = 'Сбер'
    
    def __init__(self, currency='EUR', *args, **kwargs):
        # выполняется при обращении к классу
        print('Создаем все атрибуты, необходимые для работы класса')
        self.x = 0
        self.currency = currency
        self.price = 100
        self.vat = 0.2
        
        # приватные атрибуты
        self._api_version = '5.151'  # намек на то, что менять атрибут вне кода класса опасно
        self.__secr_attr = 123  # запрет на изменение атрибута вне кода класса
    
    # функции в классах называются методы
    def method_1(self):
        # x = 1  # просто переменная
        self.x = 1  # это атрибут
        
    def method_2(self):
        self.x += 1
        print(self.x, self.currency, self.__secr_attr)
        
    @staticmethod  # декоратор
    def calculation(x, y, z):
        """Просто функции внутри классов"""
        return x**2 + y**2 + z**2
    
    @property
    def calculated_attr(self, discount=0.1):
        """Вычислимый атрибут - посчитать цену без НДС с учетом скидки discount"""
        price_w_vat = self.price / (1+self.vat)
        return price_w_vat * (1 - discount)

In [129]:
a = AnyName()

Создаем все атрибуты, необходимые для работы класса


In [131]:
a.calculated_attr

75.00000000000001

In [104]:
a.calculation(1, 2, 3)

14

In [98]:
a.method_2()

1 EUR 123


In [92]:
a.__secr_attr

AttributeError: 'AnyName' object has no attribute '__secr_attr'

In [82]:
a._api_version

'5.151'

In [86]:
a._api_version = '6.0'

In [72]:
a.BANK

'Сбер'

In [66]:
a.method_2()

1 EUR


In [24]:
a.method_1()

In [28]:
a.x = 2
a.x

2

In [30]:
a.method_2()

3


In [ ]:
datetime.strptime('2024-09-19', '%d.%m.%Y')

In [ ]:
ValueError

In [44]:
class GAParamsDateError(Exception):
    pass

In [46]:
response = {'response': 'Error: wrong date', 'status_code': 400}

In [48]:
if response['status_code'] == 400:
    raise GAParamsDateError

GAParamsDateError: 

In [16]:
def func_1(x, z):
    x += 1
    z = z * 2
    
    return x, z


def func_2(y, z):
    print(y, z)

In [10]:
res = func_1(1, 2)

In [12]:
res

(2, 4)

In [18]:
func_2(123, res[1])

123 4


### Абстрактный класс
Нужны, чтобы при наследовании родительского класса вы ОБЯЗАТЕЛЬНО переопределяли некоторые методы.

In [133]:
from abc import ABC, abstractmethod

In [135]:
class Report(ABC):
    @abstractmethod
    def sources(self):
        pass

    @abstractmethod
    def calculation(self):
        pass

    def write_to_db(self):
        # в методе запускается процесс вычислений и записи результатов
        pass

In [145]:
class SalesReport(Report):
    def sources(self):
        print('Определяем источники данных')

    def calculation(self):
        print('Пишем код вычислений')

In [147]:
class MarketingReport(Report):
    def sources(self):
        print('Определяем источники данных')

    def calculation(self):
        print('Пишем код вычислений')

In [ ]:
class MarketingEfficiencyReport(SalesReport, MarketingReport)

In [149]:
s = SalesReport()

In [151]:
s.sources()

Определяем источники данных


In [153]:
s.calculation()

Пишем код вычислений


In [155]:
s.write_to_db()

# Требования к отчету
(в хронологическом порядке)

1. Нужен отчет по трафику (визиты по дням)
2. Сделайте таблицу попроще
3. А можно сразу очень много строк получить?

[Демо-счетчик](https://metrika.yandex.ru/stat/traffic?sort=-ym%3As%3Avisits&period=2020-08-01%3A2020-09-06&accuracy=1&id=44147844&stateHash=5d3ee51607d6bde783131e11) Яндекс.Метрики


In [157]:
import requests
import json

In [159]:
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
METRIKA_ROWS_LIMIT = 5  # max 100000

DEMO_COUNTER = 44147844

In [195]:
class Metrika:
    def __init__(self, token='', counter=DEMO_COUNTER, start_date='2020-09-01', end_date='2020-09-10'):
        self.counter = counter
        self.limit = METRIKA_ROWS_LIMIT
        self.start_date = start_date
        self.end_date = end_date
        
        if token:
            self.headers = {'Authorization': 'OAuth ' + token}
        else:
            self.headers = ''  # для демо-счетчика заголовок запроса должен быть пустым
        
        self.dimensions = ''  # что мы хотим увидеть в строках отчета
        self.metrics = ''  # в столбцах
        self.report = []  # сам отчет
        
    def api_request(self, offset=1):
        """Запрос к API Метрики, возвращает словарь с отчетом"""
        params = {
            'id': self.counter,
            'date1': self.start_date,
            'date2': self.end_date,
            'metrics': self.metrics,
            'dimensions': self.dimensions,
            'limit': self.limit,
            'offset': offset,
            'accuracy': 1,
        }
        
        response = requests.get(API_URL, params=params, headers=self.headers)
        return response.json()['data']
    
    def traffic(self):
        """Отчет по посещаемости"""
        self.metrics = 'ym:s:visits'
        self.dimensions = 'ym:s:date'
        
        report = self.full_report()
        self.report = report
        
    def browsers(self):
        """Отчет по браузерам"""
        self.metrics = 'ym:s:visits'
        self.dimensions = 'ym:s:date,ym:s:browser'
        
        report = self.full_report()
        self.report = report
    
    def reformat_api_report(self):
        """Упрощение отчета до списка списков"""
        reformatted_report = []

        for line in self.report:
            dimensions = [x['name'] for x in line['dimensions']]
            reformatted_report.append(dimensions + line['metrics'])

        self.report_formatted = reformatted_report
    
    def full_report(self):
        """Постраничная выгрузка из Метрики"""
        full_data = []
        offset = 1
        
        while True:
            print('Starting offset {}'.format(offset))
            data = self.api_request(offset=offset)
            full_data += data
            
            offset += self.limit
            if not data or len(data) < self.limit:
                break
        
        return full_data

    def to_csv(self, filename):
        with open(filename, 'w') as f:
            f.write(json.dumps(self.report_formatted))

In [197]:
m = Metrika(start_date='2024-08-01', end_date='2024-09-18')

In [199]:
m.traffic()

Starting offset 1
Starting offset 6
Starting offset 11
Starting offset 16
Starting offset 21
Starting offset 26
Starting offset 31
Starting offset 36
Starting offset 41
Starting offset 46


In [201]:
m.report

[{'dimensions': [{'name': '2024-08-05'}], 'metrics': [417.0]},
 {'dimensions': [{'name': '2024-09-13'}], 'metrics': [415.0]},
 {'dimensions': [{'name': '2024-08-12'}], 'metrics': [413.0]},
 {'dimensions': [{'name': '2024-09-03'}], 'metrics': [413.0]},
 {'dimensions': [{'name': '2024-09-11'}], 'metrics': [408.0]},
 {'dimensions': [{'name': '2024-08-07'}], 'metrics': [403.0]},
 {'dimensions': [{'name': '2024-08-08'}], 'metrics': [400.0]},
 {'dimensions': [{'name': '2024-09-17'}], 'metrics': [397.0]},
 {'dimensions': [{'name': '2024-08-21'}], 'metrics': [396.0]},
 {'dimensions': [{'name': '2024-09-16'}], 'metrics': [393.0]},
 {'dimensions': [{'name': '2024-08-06'}], 'metrics': [389.0]},
 {'dimensions': [{'name': '2024-09-06'}], 'metrics': [388.0]},
 {'dimensions': [{'name': '2024-08-20'}], 'metrics': [386.0]},
 {'dimensions': [{'name': '2024-09-04'}], 'metrics': [385.0]},
 {'dimensions': [{'name': '2024-09-09'}], 'metrics': [381.0]},
 {'dimensions': [{'name': '2024-09-10'}], 'metrics': [3

In [203]:
m.reformat_api_report()
m.report_formatted

[['2024-08-05', 417.0],
 ['2024-09-13', 415.0],
 ['2024-08-12', 413.0],
 ['2024-09-03', 413.0],
 ['2024-09-11', 408.0],
 ['2024-08-07', 403.0],
 ['2024-08-08', 400.0],
 ['2024-09-17', 397.0],
 ['2024-08-21', 396.0],
 ['2024-09-16', 393.0],
 ['2024-08-06', 389.0],
 ['2024-09-06', 388.0],
 ['2024-08-20', 386.0],
 ['2024-09-04', 385.0],
 ['2024-09-09', 381.0],
 ['2024-09-10', 381.0],
 ['2024-09-05', 380.0],
 ['2024-08-13', 379.0],
 ['2024-08-26', 368.0],
 ['2024-08-14', 365.0],
 ['2024-08-22', 362.0],
 ['2024-08-30', 360.0],
 ['2024-08-29', 356.0],
 ['2024-09-12', 356.0],
 ['2024-08-15', 345.0],
 ['2024-08-19', 342.0],
 ['2024-08-09', 341.0],
 ['2024-08-28', 339.0],
 ['2024-08-27', 337.0],
 ['2024-08-16', 332.0],
 ['2024-09-02', 332.0],
 ['2024-09-18', 329.0],
 ['2024-08-23', 316.0],
 ['2024-08-02', 305.0],
 ['2024-08-11', 288.0],
 ['2024-08-01', 287.0],
 ['2024-08-03', 260.0],
 ['2024-08-10', 259.0],
 ['2024-09-07', 247.0],
 ['2024-09-15', 239.0],
 ['2024-09-14', 237.0],
 ['2024-08-24', 

In [205]:
m.browsers()
m.report

Starting offset 1
Starting offset 6
Starting offset 11
Starting offset 16
Starting offset 21
Starting offset 26
Starting offset 31
Starting offset 36
Starting offset 41
Starting offset 46
Starting offset 51
Starting offset 56
Starting offset 61
Starting offset 66
Starting offset 71
Starting offset 76
Starting offset 81
Starting offset 86
Starting offset 91
Starting offset 96
Starting offset 101
Starting offset 106
Starting offset 111
Starting offset 116
Starting offset 121
Starting offset 126
Starting offset 131
Starting offset 136
Starting offset 141
Starting offset 146
Starting offset 151
Starting offset 156
Starting offset 161
Starting offset 166
Starting offset 171
Starting offset 176
Starting offset 181
Starting offset 186
Starting offset 191
Starting offset 196
Starting offset 201
Starting offset 206
Starting offset 211
Starting offset 216
Starting offset 221
Starting offset 226
Starting offset 231
Starting offset 236
Starting offset 241
Starting offset 246
Starting offset 251
St

[{'dimensions': [{'name': '2024-08-07'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [225.0]},
 {'dimensions': [{'name': '2024-09-03'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [225.0]},
 {'dimensions': [{'name': '2024-09-17'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [221.0]},
 {'dimensions': [{'name': '2024-09-11'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [217.0]},
 {'dimensions': [{'name': '2024-09-06'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [216.0]},
 {'dimensions': [{'name': '2024-09-10'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [213.0]},
 {'dimensions': [{'name': '2024-08-20'},
   {'icon_i

In [207]:
m.reformat_api_report()
m.report_formatted

[['2024-08-07', 'Google Chrome', 225.0],
 ['2024-09-03', 'Google Chrome', 225.0],
 ['2024-09-17', 'Google Chrome', 221.0],
 ['2024-09-11', 'Google Chrome', 217.0],
 ['2024-09-06', 'Google Chrome', 216.0],
 ['2024-09-10', 'Google Chrome', 213.0],
 ['2024-08-20', 'Google Chrome', 209.0],
 ['2024-08-05', 'Google Chrome', 208.0],
 ['2024-09-09', 'Google Chrome', 206.0],
 ['2024-09-16', 'Google Chrome', 201.0],
 ['2024-08-12', 'Google Chrome', 200.0],
 ['2024-08-26', 'Google Chrome', 200.0],
 ['2024-08-06', 'Google Chrome', 199.0],
 ['2024-08-21', 'Google Chrome', 199.0],
 ['2024-09-04', 'Google Chrome', 199.0],
 ['2024-08-22', 'Google Chrome', 196.0],
 ['2024-08-13', 'Google Chrome', 194.0],
 ['2024-09-13', 'Google Chrome', 191.0],
 ['2024-08-08', 'Google Chrome', 187.0],
 ['2024-09-05', 'Google Chrome', 187.0],
 ['2024-08-27', 'Google Chrome', 185.0],
 ['2024-08-14', 'Google Chrome', 183.0],
 ['2024-08-29', 'Google Chrome', 179.0],
 ['2024-09-18', 'Google Chrome', 179.0],
 ['2024-09-12', 